In [1]:
# CBOW_1을 pytorch 그리고 첫쨰 결과값 pytorch 로 순수 미분으로 구하는것 코드 

# 솔직히 CBOW_1 제대로 이해 안됨
# 1. 평균을 안하고 걍 이어붙임
# 2. 역전파를  맨 끝에만 해서 계산함

In [2]:

sentences = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells."""



In [3]:
import re

# remove special characters and replace with ' '
# [] => 나열된 문자 혹은 범위에 해당하는 문자

# 특수문자 제거
sentences = re.sub('[^A-Za-z0-9]+', ' ', sentences)

# remove 1 letter word 첫부분 한단어,끝부분 한단어, 중간 한단어짜리 지움
# ?: non-capturing group -> 대충 매칭되도 무시하겠다 이런뜻
sentences =  re.sub(r'(?:^| )\w(?:$| )', ' ', sentences).strip()

# lower all characters
sentences = sentences.lower()

In [4]:
words = sentences.split()
vocab = set(words)
vocab_size = len(vocab)
embed_dim = 10

# sliding window 의 한쪽 면이라고 생각하는게 편한거 같다. 
context_size = 2

# implementation
word_to_idx = {word: i for i,word in enumerate(vocab)}
idx_to_word = {i:word for i,word in enumerate(vocab)}


# 변수 정리 
# vocab (43개 set) : 문장 단어set 변환
# context_size : window 크기
# word_to_idx : 단어보고 숫자 변환
# idx_to_word : 숫자보고 단어 변환
# data : [훈련시킬 데이터, 정답라벨] 리스트

In [62]:
#print(sentences)
import torch
import torch.nn as nn
import torch.optim as optim

data = []
for i in range(2, len(words)-2):
    context = [words[i-2], words[i-1], words[i+1], words[i+2]]
    target = words[i]
    data.append((context, target))

# Define the model
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.layer_1 = nn.Linear(43, 10)
        self.layer_2 = nn.Linear(10, 43)
        self.relu = nn.ReLU()
    def forward(self, *xs):
        x = sum(self.layer_1(xi) for xi in xs) / len(xs)
        x = self.relu(x)
        x = self.layer_2(x)
        return x
    
model = MyModel()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Train the model
num_epochs = 100
batch_size = 43

for epoch in range(num_epochs):
    num_correct = 0
    num_total = 0
    for context, target in data:
        inputs = torch.Tensor([word_to_idx[w] for w in context]).long()
        
        inputs_one_hot = torch.zeros(4,len(word_to_idx))
        inputs_one_hot.scatter_(1,inputs.unsqueeze(1),1)
        inputs_one_hot.unsqueeze_(1)

        labels = torch.Tensor([word_to_idx[target]]).long()

        optimizer.zero_grad()
        outputs = model(*inputs_one_hot)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        _, predicted = torch.max(outputs.data, 1)
        num_correct += (predicted == labels).sum().item()
        num_total += 1
    if epoch % 10 == 0:
        accuracy = round((num_correct / num_total) * 100,2)
        print(f"epoch : {epoch}, loss : {round(float(loss),6)} accuracy: {accuracy}%")


epoch : 0, loss : 4.384628 accuracy: 0.0%
epoch : 10, loss : 3.457874 accuracy: 59.26%
epoch : 20, loss : 0.796906 accuracy: 98.15%
epoch : 30, loss : 0.147473 accuracy: 100.0%
epoch : 40, loss : 0.057648 accuracy: 100.0%
epoch : 50, loss : 0.028062 accuracy: 100.0%
epoch : 60, loss : 0.01623 accuracy: 100.0%
epoch : 70, loss : 0.009981 accuracy: 100.0%
epoch : 80, loss : 0.006415 accuracy: 100.0%
epoch : 90, loss : 0.004339 accuracy: 100.0%
